<a href="https://colab.research.google.com/github/yev34/Machine-learning/blob/main/%D0%9F%D0%B0%D1%82%D0%B0%D0%BB%D0%B0%D1%88%D0%BA%D0%BE_%D0%BB%D0%B0%D0%B1_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357286 sha256=cc8075d49d9b5c002d403c6e1d83b983d458f82efca2aa038e86ccedc58dbeec
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset, Reader
from surprise import SVD, KNNBasic
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split

Завантажте датасет для рецензій (ml-100k):

In [ ]:


data = Dataset.load_builtin('ml-100k')

Виведіть перші 5 рядків:

In [ ]:
df = data.build_full_trainset().build_anti_testset()
print("Перші 5 рецензій:")
for i in range(5):
    print(df[i])

Перші 5 рецензій:
('196', '302', 3.52986)
('196', '377', 3.52986)
('196', '51', 3.52986)
('196', '346', 3.52986)
('196', '474', 3.52986)


Реалізуйте два алгоритми для рекомендаційної системи:

Використайте крос-валідацію для підбору параметрів:

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2)

param_grid_svd = {
    'n_factors': [50, 100],
    'lr_all': [0.005, 0.010],
    'reg_all': [0.02, 0.05]
}

param_grid_knn = {
    'k': [20, 40],
    'min_k': [1, 5],
    'sim_options': {
        'name': ['cosine', 'pearson'],
        'user_based': [False, True]
    }
}

grid_search_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=5)
grid_search_svd.fit(data)

grid_search_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['rmse', 'mae'], cv=5)
grid_search_knn.fit(data)


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing

Оберіть найкращий алгоритм:

In [ ]:

best_svd = grid_search_svd.best_estimator['mae']
best_knn = grid_search_knn.best_estimator['mae']

print(f"Найкращий MAE для SVD: {grid_search_svd.best_score['mae']}")
print(f"Найкращий MAE для KNNBasic: {grid_search_knn.best_score['mae']}")

if grid_search_svd.best_score['mae'] < grid_search_knn.best_score['mae']:
    best_model = best_svd
    print("Вибраний алгоритм: SVD")
else:
    best_model = best_knn
    print("Вибраний алгоритм: KNNBasic")

Найкращий MAE для SVD: 0.724246357041128
Найкращий MAE для KNNBasic: 0.8037707143269521
Вибраний алгоритм: SVD


Вивести рекомендації для конкретного користувача:

In [ ]:
best_model.fit(trainset)

user_id = str(196)
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
print(f"Кількість рецензій користувача {user_id}: {len(user_ratings)}")

all_items = set(trainset.all_items())
rated_items = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

predictions = [(item, best_model.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

print("Топ-10 фільмів, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")


Кількість рецензій користувача 196: 33
Топ-10 фільмів, рекомендованих для користувача:
Фільм 64 з прогнозованим рейтингом 4.52
Фільм 313 з прогнозованим рейтингом 4.47
Фільм 50 з прогнозованим рейтингом 4.44
Фільм 483 з прогнозованим рейтингом 4.44
Фільм 30 з прогнозованим рейтингом 4.39
Фільм 169 з прогнозованим рейтингом 4.38
Фільм 22 з прогнозованим рейтингом 4.37
Фільм 302 з прогнозованим рейтингом 4.35
Фільм 172 з прогнозованим рейтингом 4.35
Фільм 191 з прогнозованим рейтингом 4.35
